In [1]:
import pandas as pd
import numpy as np
from lr.text_processing.util import simple_pre_process_text_df
from multiprocessing import Pool
from functools import reduce
from collections import Counter

In [2]:
syn_noum = pd.read_csv("data/snli/syn_noun.csv")
keys = syn_noum.key.to_list()
values = syn_noum.value.to_list()
syn_noum_dict = {k:v for k,v in zip(syn_noum.key, syn_noum.value)}
syn_noum_dict_inv = {v:k for k,v in zip(syn_noum.key, syn_noum.value)}

def check_key(s):
    return [w for w in s.split(" ") if w in keys] 

def check_key_df(df):    
    df.loc[:, "w_trans"] = df["text"].map(check_key)
    df.loc[:, "n_trans"] = df["w_trans"].map(lambda x : len(x))
    return df
    
def parallelize(df, func, n_cores):
    """
    general fucntion to parallelize a function applied to
    a df
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
df = pd.read_csv("data/snli/test.csv").dropna()
simple_pre_process_text_df(df, "premise")
simple_pre_process_text_df(df, "hypothesis")
df.loc[:, "text"] = df.premise + " " + df.hypothesis
df.loc[:, "size"] = df["text"].map(lambda x: len(x.split(" ")))
df = parallelize(df, func=check_key_df, n_cores=20)

df_neu  = df.query("label=='neutral'")
df_con  = df.query("label=='contradiction'")
df_ent  = df.query("label=='entailment'")

df_p = pd.read_csv("data/snli/test_p_h_syn_noun.csv").dropna()
df_p.loc[:, "text"] = df_p.premise + " " + df_p.hypothesis
df_p.loc[:, "size"] = df_p["text"].map(lambda x: len(x.split(" ")))
df_p = parallelize(df_p, func=check_key_df, n_cores=20)

df_p_neu  = df_p.query("label=='neutral'")
df_p_con  = df_p.query("label=='contradiction'")
df_p_ent  = df_p.query("label=='entailment'")

## Sizes (Original)

In [4]:
avg_total = df[["size"]].describe().loc["mean"].values[0]
avg_con = df_con[["size"]].describe().loc["mean"].values[0]
avg_neu = df_neu[["size"]].describe().loc["mean"].values[0]
avg_ent = df_ent[["size"]].describe().loc["mean"].values[0]
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
original = pd.DataFrame([results], columns=columns)
original

,total,contradiction,neutral,entailment
0,21.44,21.2,22.15,20.85


## Sizes (Transformed)

In [5]:
avg_total = df_p[["size"]].describe().loc["mean"].values[0]
avg_con = df_p_con[["size"]].describe().loc["mean"].values[0]
avg_neu = df_p_neu[["size"]].describe().loc["mean"].values[0]
avg_ent = df_p_ent[["size"]].describe().loc["mean"].values[0]
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
transformed = pd.DataFrame([results], columns=columns)
transformed

,total,contradiction,neutral,entailment
0,24.8,24.64,25.6,24.2


In [6]:
pd.concat([original, transformed]).pct_change()* 100

,total,contradiction,neutral,entailment
0,NaN,NaN,NaN,NaN
0,15.671642,16.226415,15.575621,16.067146


In [7]:
df.head(3)

,premise,hypothesis,label,text,size,w_trans,n_trans
0,this church choir sings to the masses as they ...,the church has cracks in the ceiling,neutral,this church choir sings to the masses as they ...,25,"[church, masses, church, church]",4
1,this church choir sings to the masses as they ...,the church is filled with song,entailment,this church choir sings to the masses as they ...,24,"[church, masses, church, church, song]",5
2,this church choir sings to the masses as they ...,a choir singing at a baseball game,contradiction,this church choir sings to the masses as they ...,25,"[church, masses, church, baseball]",4


## Count transformations

In [8]:
avg_total = df["n_trans"].mean()
avg_con = df_con["n_trans"].mean()
avg_neu = df_neu["n_trans"].mean()
avg_ent = df_ent["n_trans"].mean()
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
n_tranforms = pd.DataFrame([results], columns=columns)
n_tranforms

,total,contradiction,neutral,entailment
0,3.42,3.39,3.53,3.32


## Check transformations

In [9]:
def lists2list(arr):
    return reduce(lambda x,y: x+y, arr)

def get_counter(arrays, n_cores):
    splits = np.array_split(arrays, n_cores)
    pool = Pool(n_cores)
    final_list = lists2list(pool.map(lists2list, splits))
    pool.close()
    pool.join()
    key_count = Counter(final_list)
    return key_count

count_total = get_counter(df["w_trans"].values, n_cores=20)
count_total.most_common(10)

[('man', 5076),
 ('woman', 2610),
 ('boy', 1214),
 ('dog', 907),
 ('person', 778),
 ('child', 560),
 ('children', 489),
 ('bike', 292),
 ('snow', 290),
 ('picture', 261)]

In [10]:
count_con = get_counter(df_con["w_trans"].values, n_cores=20)
count_con.most_common(10)

[('man', 1661),
 ('woman', 852),
 ('boy', 404),
 ('dog', 323),
 ('person', 221),
 ('child', 174),
 ('children', 152),
 ('bike', 90),
 ('park', 89),
 ('snow', 89)]

In [11]:
count_neu = get_counter(df_neu["w_trans"].values, n_cores=20)
count_neu.most_common(10)

[('man', 1635),
 ('woman', 851),
 ('boy', 389),
 ('dog', 264),
 ('person', 215),
 ('child', 168),
 ('children', 165),
 ('bike', 103),
 ('snow', 96),
 ('couple', 82)]

In [12]:
count_ent = get_counter(df_ent["w_trans"].values, n_cores=20)
count_ent.most_common(10)

[('man', 1680),
 ('woman', 874),
 ('boy', 385),
 ('person', 327),
 ('dog', 306),
 ('child', 208),
 ('children', 167),
 ('outdoors', 113),
 ('picture', 107),
 ('snow', 103)]

## Artifacts analysis

In [13]:
count_gen = lambda w: lambda s: int(w in s.split(" "))
con_art = ["nobody", "sleeping", "no", "tv", "cat"]
neu_art = ["tall", "first", "competition", "sad", "favorite"]
ent_art = ["outdoors", "least", "instrument", "outside", "animal"]
con_art_t = [(w, syn_noum_dict[w]) for w in con_art if w in keys]
neu_art_t = [(w, syn_noum_dict[w]) for w in neu_art if w in keys]
ent_art_t = [(w, syn_noum_dict[w]) for w in ent_art if w in keys]

### Countradition

In [14]:
con_art_t, [(syn_noum_dict_inv[w], w) for w in con_art if w in values]

([('tv', 'video'), ('cat', 'true cat')], [])

In [15]:
for w in con_art:
    f_count = count_gen(w)
    df_con.loc[:, "{}_count".format(w)] = df_con["text"].map(f_count)
    df_p_con.loc[:, "{}_count".format(w)] = df_p_con["text"].map(f_count)
    print(w, np.round(df_con["{}_count".format(w)].mean()*100,2), np.round(df_p_con["{}_count".format(w)].mean()*100,2))

nobody 1.39 1.39
sleeping 3.55 3.55
no 13.13 13.19
tv 1.02 0.09
cat 2.78 3.4


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


### Neutral

In [16]:
neu_art_t, [(syn_noum_dict_inv[w], w) for w in neu_art if w in values]

([('favorite', 'favourite')], [('1st', 'first'), ('rivals', 'competition')])

In [17]:
for w in neu_art:
    f_count = count_gen(w)
    df_neu.loc[:, "{}_count".format(w)] = df_neu["text"].map(f_count)
    df_p_neu.loc[:, "{}_count".format(w)] = df_p_neu["text"].map(f_count)
    print(w, np.round(df_neu["{}_count".format(w)].mean()*100,2), np.round(df_p_neu["{}_count".format(w)].mean()*100,2))

tall 1.71 2.33
first 0.75 0.78
competition 0.81 1.06
sad 0.47 0.47
favorite 0.16 0.0


### Entailment

In [18]:
ent_art_t, [(syn_noum_dict_inv[w], w) for w in ent_art if w in values]

([('outdoors', 'open'), ('animal', 'creature')],
 [('outdoor', 'outside'), ('creatures', 'animal')])

In [19]:
for w in ent_art:
    f_count = count_gen(w)
    df_ent.loc[:, "{}_count".format(w)] = df_ent["text"].map(f_count)
    df_p_ent.loc[:, "{}_count".format(w)] = df_p_ent["text"].map(f_count)
    print(w, np.round(df_ent["{}_count".format(w)].mean()*100,2), np.round(df_p_ent["{}_count".format(w)].mean()*100,2))

outdoors 3.27 0.0
least 0.24 0.24
instrument 1.07 1.07
outside 9.47 10.04
animal 1.72 0.74
